In [40]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

import os
import random

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
from torch.utils.mobile_optimizer import optimize_for_mobile

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Load Data

In [41]:
def loadData():
    dirPath = os.getcwd()
    acce_columns = ['time', 'acce_x', 'acce_y', 'acce_z']
    linacce_columns = ['time', 'linacce_x', 'linacce_y', 'linacce_z']
    gyro_columns = ['time', 'gyro_x', 'gyro_y', 'gyro_z']
    
    allData = list()
    
    for dataDir in os.listdir(dirPath + "/allData"):
        path = dirPath + "/allData/" + dataDir + "/"
        acce_columns = ['time', 'acce_x', 'acce_y', 'acce_z']
        gyro_columns = ['time', 'gyro_x', 'gyro_y', 'gyro_z']
        if (dataDir.split('_'))[1] == "walk":
            lable = 1
        else:
            lable = 0
        
        df_acce = pd.read_csv(path+"acce.txt", skiprows=1, names=acce_columns, delim_whitespace=True)
        df_acce = df_acce.drop("time", axis=1)
        df_gyro = pd.read_csv(path+"gyro.txt", skiprows=1, names=gyro_columns, delim_whitespace=True)
        df_gyro = df_gyro.drop("time", axis=1)
        df_linacce = pd.read_csv(path+"linacce.txt", skiprows=1, names=linacce_columns, delim_whitespace=True)
        df_linacce = df_linacce.drop("time", axis=1)

        df_new = pd.concat([df_acce, df_gyro, df_linacce], axis=1)
        df_new["label"] = lable
        allData.append(df_new)
    
    return pd.concat(allData, ignore_index=True)

In [42]:
data = loadData()

/tmp/ipykernel_6893/2576324855.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_acce = pd.read_csv(path+"acce.txt", skiprows=1, names=acce_columns, delim_whitespace=True)
/tmp/ipykernel_6893/2576324855.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_gyro = pd.read_csv(path+"gyro.txt", skiprows=1, names=gyro_columns, delim_whitespace=True)
/tmp/ipykernel_6893/2576324855.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_linacce = pd.read_csv(path+"linacce.txt", skiprows=1, names=linacce_columns, delim_whitespace=True)
/tmp/ipykernel_6893/2576324855.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep

In [43]:
data.fillna(data.mean(), inplace=True)
X = data.drop(columns=['label'])
y = data['label']

In [44]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Prepare Data

In [45]:
def create_sequences(data, labels, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = labels[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [46]:
sequence_length = 10
X_sequences, y_sequences = create_sequences(X, y, sequence_length)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

In [48]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## Create Model

In [49]:
class WalkDetectionRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(WalkDetectionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)

## Train

In [50]:
input_size = X_train.shape[2]
hidden_size = 64
output_size = 1
num_layers = 2

In [51]:
model = WalkDetectionRNN(input_size, hidden_size, output_size, num_layers)

In [52]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [53]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor)
    optimizer.zero_grad()
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 0.6251
Epoch [20/200], Loss: 0.5417
Epoch [30/200], Loss: 0.4888
Epoch [40/200], Loss: 0.3730
Epoch [50/200], Loss: 0.2352
Epoch [60/200], Loss: 0.1702
Epoch [70/200], Loss: 0.1370
Epoch [80/200], Loss: 0.1007
Epoch [90/200], Loss: 0.0625
Epoch [100/200], Loss: 0.0378
Epoch [110/200], Loss: 0.0266
Epoch [120/200], Loss: 0.0206
Epoch [130/200], Loss: 0.0163
Epoch [140/200], Loss: 0.0132
Epoch [150/200], Loss: 0.0109
Epoch [160/200], Loss: 0.0092
Epoch [170/200], Loss: 0.0079
Epoch [180/200], Loss: 0.0069
Epoch [190/200], Loss: 0.0060
Epoch [200/200], Loss: 0.0052


## Test

In [55]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)
    predicted = (test_outputs.squeeze() > 0.5).float()
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test Loss: {test_loss.item():.4f}')

Test Accuracy: 0.9980
Test Loss: 0.0106


## Save model

In [60]:
traced_script_module = torch.jit.trace(model, X_test_tensor)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("model.ptl")